In [174]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [175]:
y = np.loadtxt("d:/datasets/uci/gisette/gisette_train.labels", delimiter=',')
x = np.loadtxt("d:/datasets/uci/gisette/gisette_train.data")
yt = np.loadtxt("d:/datasets/uci/gisette/gisette_valid.labels", delimiter=',')
xt = np.loadtxt("d:/datasets/uci/gisette/gisette_valid.data")

In [176]:
mx=np.mean(x,axis=0)
sx=np.std(x,axis=0)
sx[sx==0]=0.00001
x=(x-mx)/sx
n,d=x.shape
X=x.copy()
xt=(xt-mx)/sx
#yt[yt<0]=0
yt=yt[:, np.newaxis]
print(X.shape,yt.shape)

(6000, 5000) (1000, 1)


In [177]:
y1=y.copy()
y1=y1[:, np.newaxis]
y=y[:, np.newaxis]
y[y<0]=0
print(y)
print(y1)

[[1.]
 [0.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]
[[ 1.]
 [-1.]
 [ 1.]
 ...
 [-1.]
 [-1.]
 [-1.]]


In [178]:
def hval(x,idx,b,b0):
    if not idx:
        n=x.shape[0]
        h=np.zeros((n))+b0
    else:
        h=np.matmul(x[:,idx],b)+b0
    return h
def err(u,y):
    y1=torch.argmax(u,1)
    return torch.mean((y1!=y).float())
def fitRegressors(x,z,w):
    sw=np.sum(w)
    xbar=np.sum(x*w,axis=0)/sw
    ybar=np.sum(z*w)/sw
    s1=np.sum((x-xbar)*y*w,axis=0)
    s2=np.sum(w*(x-xbar)**2,axis=0)
    b1=s1/(s2+0.00001)
    b0=ybar-b1*xbar
    return b1,b0

def losses(x,y,h0,b1,b0):
    h= x*b1+b0+h0
    yxw=y*h
    l=np.log(1+np.exp(-yxw))
    l[yxw<0]=-yxw[yxw<0]+np.log(1+np.exp(yxw[yxw<0]))
    lo=np.mean(l,axis=0)
    return lo
def loss(h,y):
    yxw=y*h
    l=np.log(1+np.exp(-yxw))
    #print(l.shape)
    l[yxw<0]=-yxw[yxw<0]+np.log(1+np.exp(yxw[yxw<0]))
    lo=np.mean(l)
    #print(lo.shape)
    return lo

In [179]:
k=20
idx=[]
b=[]
b00=0
for i in range(k):
    bb=np.array(b)
    h=hval(x,idx,b,b00)
    h=h[:, np.newaxis]
    #print(h.shape)    
    p=1/(1+np.exp(-h))
    w=p*(1-p)
    z=1/p
    z[y==0]=-1/(1-p[y==0])
    b1,b0=fitRegressors(x,z,w)
    l=losses(x,y1,h,b1,b0)
    j=np.argmin(l)
    print(i,j,l[j])
    idx.append(j)
    b.append(b1[j])
    b00+=b0[j]

0 3656 0.5944739437106029
1 3656 0.5229153184618474
2 557 0.47107283310099424
3 904 0.4351975378151704
4 3975 0.40810475428722254
5 3065 0.38689600572264327
6 3543 0.37033687995367925
7 4507 0.35626878801247547
8 3062 0.3455548954505691
9 3251 0.33484216069869
10 4424 0.32475799071728645
11 2740 0.3164529904529219
12 467 0.3075191684061645
13 4158 0.2995071152452865
14 1277 0.29170424448758514
15 2488 0.2842086951706291
16 4424 0.2781685394563331
17 4553 0.27186573642871875
18 288 0.2658937192459419
19 732 0.2604061592242541
